In [1]:
# Import Packages
import pandas as pd
from patsy import dmatrices
import numpy as np
import statsmodels.api as sm
import matplotlib.pyplot as plt
import datetime
from statsmodels.tsa.api import VAR
import matplotlib.pyplot as plt

In [2]:
# Read in shark data
eat = pd.read_excel("SharkData.xlsx", sheet_name = 0)
targets = pd.read_excel("SharkData.xlsx", sheet_name = 1)
drops = pd.read_excel("SharkData.xlsx", sheet_name = 2)
other_factors = pd.read_excel("SharkData.xlsx", sheet_name = 3)

# Data Cleaning

In [3]:
# Clean eating dataset
eat = eat.iloc[1:,:-1]
eat.rename(columns = {"Unnamed: 14":'Etc. Comments', "Pieces Eaten": "Date", 'Total: ' : 'Total'}, inplace = True)

#get rid of null values at end of dataset
eat = eat.head(582)

# Change date to just date format, not datetime
eat['Date'] = pd.to_datetime(eat['Date']).dt.date



eat.reset_index(drop=True,inplace = True)
eat.head()

,Date,Ross,Chandler,BT1,BT2,BT3,BT4,BT5,GR1,GR2,GR3,GR4,GR5,Total,Etc. Comments
0,2017-12-05,0.0,5.0,3.0,6.0,5.0,4.0,4.0,0.0,0.0,2.0,3.0,1.0,33.0,NaN
1,2017-12-07,1.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,NaN
2,2017-12-09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,0.0,3.0,1.0,0.0,6.0,NaN
3,2017-12-10,5.0,1.0,1.0,3.0,2.0,2.0,0.0,NaN,NaN,NaN,NaN,NaN,14.0,NaN
4,2017-12-12,2.0,2.0,0.0,5.0,2.0,4.0,5.0,0.0,2.0,3.0,2.0,0.0,27.0,NaN


In [4]:
# Clean Drops Dataset
drops = drops.iloc[1:,:-1]
drops['Drops'] = pd.to_datetime(drops['Drops']).dt.date
drops.rename(columns = {"Drops" : "Date", "Total: ":"Total"}, inplace = True)
drops.reset_index(drop=True,inplace = True)
drops.tail()

,Date,Ross,Chandler,BT1,BT2,BT3,BT4,BT5,GR1,GR2,GR3,GR4,GR5,Total
577,2020-09-22,0.0,0.0,1.0,4.0,0.0,2.0,2.0,0.0,0.0,0.0,1.0,0.0,10.0
578,2020-09-24,0.0,0.0,0.0,3.0,1.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,7.0
579,2020-09-26,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,1.0,1.0
580,2020-09-27,0.0,0.0,2.0,0.0,0.0,0.0,1.0,NaN,NaN,NaN,NaN,NaN,3.0
581,2020-09-29,0.0,0.0,0.0,2.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,4.0


In [5]:
# Clean Targets Dataset
targets = targets.iloc[1:,:-1]
targets.rename(columns = {"Unnamed: 14":'Etc. Comments', "Targets": "Date", "Total: ": "Total"}, inplace = True)

#get rid of null values at end of dataset
targets = targets.head(582)

# Change date to just date format, not datetime
targets['Date'] = pd.to_datetime(targets['Date']).dt.date

targets.reset_index(drop=True,inplace = True)
targets.tail()

,Date,Ross,Chandler,BT1,BT2,BT3,BT4,BT5,GR1,GR2,GR3,GR4,GR5,Total
577,2020-09-22,6.0,1.0,4.0,6.0,5.0,4.0,7.0,2.0,1.0,5.0,4.0,1.0,46.0
578,2020-09-24,8.0,18.0,2.0,9.0,10.0,10.0,11.0,5.0,6.0,3.0,7.0,9.0,98.0
579,2020-09-26,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6.0,2.0,3.0,5.0,8.0,24.0
580,2020-09-27,10.0,19.0,9.0,10.0,8.0,16.0,3.0,NaN,NaN,NaN,NaN,NaN,75.0
581,2020-09-29,15.0,5.0,6.0,6.0,4.0,4.0,7.0,4.0,1.0,4.0,4.0,1.0,61.0


# Other Factors Data Cleaning and Dummy Variable Creation for Foods


In [6]:
dummy_factors = other_factors[['Saury', 'Blue Runner',
                'Squid', 'Mackerel', 'Herring', 'Sardine',
                'Mazuri Vitamins', 'Garlic', 'Salmon', 'Bonito', 'Bluefish', 'Mahi',
                    'Goggle Eye', 'Humbolt Squid']]
other_dummies = other_factors[['BT/SB Location (1-6)','GR Location (1-6)']]
other_dummies.rename(columns= {"BT/SB Location (1-6)": "BT_SB_Location", "GR Location (1-6)": "GR_Location"}, inplace = True)
for i in dummy_factors:
    dummy_factors[i] = dummy_factors[i].fillna(0)
dummy_factors.rename(columns = {"Blue Runner": "Blue_Runner", "Mazuri Vitamins": "Mazuri_Vitamins", 
                               "Goggle Eye": "Goggle_Eye", "Humbolt Squid": "Humbolt_Squid"}, inplace = True)

/home/sport10/.local/lib/python3.8/site-packages/pandas/core/frame.py:4295: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(
<ipython-input-6-456bd9ea25b4>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dummy_factors[i] = dummy_factors[i].fillna(0)


# Combine Datasets to get all factors to perform regression on

In [7]:
GroupFeed = []
for row in drops.index:
    if drops.iloc[row,:].isna().sum() > 0:
        GroupFeed.append(0)
    else:
        GroupFeed.append(1)
GroupFeed

#you could do something like this to add it to a dataframe
#eat["GroupFeed"] = GroupFeed


[1,
 1,
 0,
 0,
 1,
 1,
 0,
 0,
 1,
 1,
 0,
 0,
 1,
 1,
 0,
 0,
 1,
 1,
 0,
 0,
 1,
 1,
 0,
 0,
 1,
 1,
 0,
 0,
 1,
 1,
 0,
 0,
 1,
 1,
 0,
 0,
 1,
 1,
 0,
 0,
 1,
 1,
 0,
 0,
 1,
 1,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 1,
 1,
 0,
 0,
 1,
 1,
 0,
 0,
 1,
 1,
 0,
 0,
 1,
 1,
 0,
 0,
 1,
 1,
 0,
 0,
 1,
 1,
 0,
 0,
 1,
 1,
 0,
 0,
 1,
 1,
 0,
 0,
 1,
 1,
 0,
 0,
 1,
 1,
 0,
 0,
 1,
 1,
 0,
 0,
 1,
 1,
 0,
 0,
 1,
 1,
 0,
 0,
 1,
 1,
 0,
 0,
 1,
 1,
 0,
 0,
 1,
 1,
 0,
 0,
 1,
 1,
 0,
 0,
 1,
 1,
 0,
 0,
 1,
 1,
 0,
 0,
 1,
 1,
 0,
 0,
 1,
 1,
 0,
 0,
 1,
 1,
 0,
 0,
 1,
 1,
 0,
 0,
 1,
 1,
 0,
 0,
 1,
 1,
 0,
 0,
 1,
 1,
 0,
 1,
 1,
 1,
 0,
 0,
 1,
 1,
 0,
 0,
 1,
 1,
 0,
 0,
 1,
 1,
 0,
 0,
 1,
 1,
 0,
 0,
 1,
 1,
 0,
 0,
 1,
 1,
 0,
 0,
 1,
 1,
 0,
 0,
 1,
 1,
 0,
 0,
 1,
 1,
 0,
 0,
 1,
 0,
 0,
 1,
 1,
 0,
 0,
 1,
 1,
 0,
 0,
 1,
 1,
 0,
 0,
 1,
 1,
 0,
 0,
 1,
 0,
 0,
 1,
 1,
 0,
 0,
 1,
 1,
 0,
 0,
 1,
 1,
 0,
 0,
 1,
 1,
 0,
 0,
 1,
 1,
 0,
 0,
 1,
 1,
 0,
 0,
 1,
 1,
 0,
 0,
 1,


In [8]:
# combine eat data
eat_concat1 = pd.concat([eat,dummy_factors,other_dummies], axis = 1)
eat_concat = pd.concat([eat_concat1, other_factors["Temperature"]], axis = 1)
eat_concat["GroupFeed"] = GroupFeed
eat_concat.head()

,Date,Ross,Chandler,BT1,BT2,BT3,BT4,BT5,GR1,GR2,...,Salmon,Bonito,Bluefish,Mahi,Goggle_Eye,Humbolt_Squid,BT_SB_Location,GR_Location,Temperature,GroupFeed
0,2017-12-05,0.0,5.0,3.0,6.0,5.0,4.0,4.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,3.0,73.2,1
1,2017-12-07,1.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,3.0,73.2,1
2,2017-12-09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,NaN,3.0,73.6,0
3,2017-12-10,5.0,1.0,1.0,3.0,2.0,2.0,0.0,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,NaN,73.6,0
4,2017-12-12,2.0,2.0,0.0,5.0,2.0,4.0,5.0,0.0,2.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,3.0,73.4,1


In [9]:
# Add eat variable for Covid shut down
start_date = pd.to_datetime('03-14-2020')
end_date = pd.to_datetime('05-09-2020')
conditions = [ (eat_concat['Date'] >= start_date) & (eat_concat['Date'] <= end_date),
              (eat_concat['Date'] < start_date) | (eat_concat['Date'] > end_date)]
values = [1, 0]
eat_concat["covid"] = np.select(conditions, values)

In [10]:
# Add eat variable for light training
light_start = pd.to_datetime('02-15-2018')
conditions_light = [ (eat_concat['Date'] >= light_start), (eat_concat['Date'] < light_start)]
values_light = [1, 0]
eat_concat["light_training"] = np.select(conditions_light, values_light)

In [11]:
# combine drops data
drops_concat1 = pd.concat([drops,dummy_factors,other_dummies], axis = 1)
drops_concat = pd.concat([drops_concat1, other_factors["Temperature"]], axis = 1)
drops_concat["GroupFeed"] = GroupFeed
drops_concat.head()

,Date,Ross,Chandler,BT1,BT2,BT3,BT4,BT5,GR1,GR2,...,Salmon,Bonito,Bluefish,Mahi,Goggle_Eye,Humbolt_Squid,BT_SB_Location,GR_Location,Temperature,GroupFeed
0,2017-12-05,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,3.0,73.2,1
1,2017-12-07,0.0,1.0,1.0,2.0,1.0,1.0,2.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,3.0,73.2,1
2,2017-12-09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,NaN,3.0,73.6,0
3,2017-12-10,0.0,0.0,0.0,0.0,0.0,0.0,1.0,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,NaN,73.6,0
4,2017-12-12,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,3.0,73.4,1


In [12]:
# Add drop variable for Covid shut down
start_date = pd.to_datetime('03-14-2020')
end_date = pd.to_datetime('05-09-2020')
conditions = [ (drops_concat['Date'] >= start_date) & (drops_concat['Date'] <= end_date),
              (drops_concat['Date'] < start_date) | (drops_concat['Date'] > end_date)]
values = [1, 0]
drops_concat["covid"] = np.select(conditions, values)

In [13]:
# Add drop variable for light training
light_start = pd.to_datetime('02-15-2018')
conditions_light = [ (drops_concat['Date'] >= light_start), (drops_concat['Date'] < light_start)]
values_light = [1, 0]
drops_concat["light_training"] = np.select(conditions_light, values_light)

In [14]:
# combine Target data
targets_concat1 = pd.concat([targets,dummy_factors,other_dummies], axis = 1)
targets_concat = pd.concat([targets_concat1, other_factors["Temperature"]], axis = 1)
targets_concat["GroupFeed"] = GroupFeed
targets_concat.head()

,Date,Ross,Chandler,BT1,BT2,BT3,BT4,BT5,GR1,GR2,...,Salmon,Bonito,Bluefish,Mahi,Goggle_Eye,Humbolt_Squid,BT_SB_Location,GR_Location,Temperature,GroupFeed
0,2017-12-05,3.0,0.0,4.0,3.0,0.0,2.0,2.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,3.0,73.2,1
1,2017-12-07,4.0,5.0,2.0,2.0,5.0,3.0,5.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,3.0,73.2,1
2,2017-12-09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,3.0,...,0.0,0.0,0.0,0.0,0.0,0.0,NaN,3.0,73.6,0
3,2017-12-10,4.0,7.0,0.0,2.0,2.0,0.0,0.0,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,NaN,73.6,0
4,2017-12-12,3.0,1.0,3.0,2.0,4.0,1.0,1.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,3.0,73.4,1


In [15]:
# add column to dataset that indicates whether targets were being varied or not
varied_start1 = pd.to_datetime('04-01-2018')
varied_stop = pd.to_datetime('05-12-2018')
varied_start2 = pd.to_datetime('07-26-2018')

eat_concat["Varied_Target"] = [1 if (varied_start1 <= ele <= varied_stop) | (ele >= varied_start2)  else 0 for ele in eat_concat["Date"]]
drops_concat["Varied_Target"] = [1 if (varied_start1 <= ele <= varied_stop) | (ele >= varied_start2)  else 0 for ele in drops_concat["Date"]]
targets_concat["Varied_Target"] = [1 if (varied_start1 <= ele <= varied_stop) | (ele >= varied_start2)  else 0 for ele in targets_concat["Date"]]

In [16]:
# Add target variable for Covid shut down
start_date = pd.to_datetime('03-14-2020')
end_date = pd.to_datetime('05-09-2020')
conditions = [ (targets_concat['Date'] >= start_date) & (targets_concat['Date'] <= end_date),
              (targets_concat['Date'] < start_date) | (targets_concat['Date'] > end_date)]
values = [1, 0]
targets_concat["covid"] = np.select(conditions, values)

In [17]:
# Add target variable for light training
light_start = pd.to_datetime('02-15-2018')
conditions_light = [ (targets_concat['Date'] >= light_start), (targets_concat['Date'] < light_start)]
values_light = [1, 0]
targets_concat["light_training"] = np.select(conditions_light, values_light)

# Subset Creation

In [18]:

# All SS Subset
All_SS = eat_concat[['Ross', 'Chandler']]

# All BT Subset
All_BT = eat_concat[['BT1', 'BT2', 'BT3', 'BT4', 'BT5']]

# All GR Subset
All_GR = eat_concat[['GR1', 'GR2', 'GR3', 'GR4', 'GR5']]

# Male Subset
Male = eat_concat[["BT1","BT5","GR1","Ross","Chandler"]]

# Female Subset
female = eat_concat[["BT2","BT3","BT4","GR2","GR3","GR4","GR5"]]

# Append all subsets to original datasets
# eat dataset
eat_concat["All_GR"] = eat['GR1'] + eat['GR2'] + eat['GR3'] + eat['GR4'] + eat['GR5']
eat_concat["All_BT"] = eat['BT1'] + eat['BT2'] + eat['BT3'] + eat['BT4'] + eat['BT5']
eat_concat["All_SS"] = eat['Ross'] + eat['Chandler']
eat_concat["male"] = eat[["BT1","BT5","GR1","Ross","Chandler"]].sum(axis = 1)
eat_concat["female"] = eat[["BT2","BT3","BT4","GR2","GR3","GR4","GR5"]].sum(axis = 1)

# drop dataset
drops_concat["All_GR"] = drops['GR1'] + drops['GR2'] + drops['GR3'] + drops['GR4'] + drops['GR5']
drops_concat["All_BT"] = drops['BT1'] + drops['BT2'] + drops['BT3'] + drops['BT4'] + drops['BT5']
drops_concat["All_SS"] = drops['Ross'] + drops['Chandler']
drops_concat["male"] = drops[["BT1","BT5","GR1","Ross","Chandler"]].sum(axis = 1)
drops_concat["female"] = drops[["BT2","BT3","BT4","GR2","GR3","GR4","GR5"]].sum(axis = 1)

# target dataset
targets_concat["All_GR"] = targets['GR1'] + targets['GR2'] + targets['GR3'] + targets['GR4'] + targets['GR5']
targets_concat["All_BT"] = targets['BT1'] + targets['BT2'] + targets['BT3'] + targets['BT4'] + targets['BT5']
targets_concat["All_SS"] = targets['Ross'] + targets['Chandler']
targets_concat["male"] = targets[["BT1","BT5","GR1","Ross","Chandler"]].sum(axis = 1)
targets_concat["female"] = targets[["BT2","BT3","BT4","GR2","GR3","GR4","GR5"]].sum(axis = 1)


In [19]:
eat_concat = eat_concat.drop(columns = ["BT_SB_Location","GR_Location"])
drops_concat = drops_concat.drop(columns = ["BT_SB_Location","GR_Location"])
targets_concat = targets_concat.drop(columns = ["BT_SB_Location","GR_Location"])

# Create Tables for Poisson Regression Time Series For Pieces Eaten For All Sharks

In [20]:
# Create training and testing datasets
mask = np.random.rand(len(eat_concat)) < 0.8
eat_train = eat_concat[mask]
eat_test = eat_concat[~mask]
print('Training data set length='+str(len(eat_train)))
print('Testing data set length='+str(len(eat_test)))

Training data set length=472
Testing data set length=110


In [161]:
pd.concat([eat_concat.iloc[:,14:-6].drop(columns = "Etc. Comments"),eat_concat["Temp_diff"]],axis = 1)

,Saury,Blue_Runner,Squid,Mackerel,Herring,Sardine,Mazuri_Vitamins,Garlic,Salmon,Bonito,Bluefish,Mahi,Goggle_Eye,Humbolt_Squid,Temperature,GroupFeed,covid,light_training,Varied_Target,Temp_diff
0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,73.2,1,0,0,0,0.0
1,0.0,1.0,1.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,73.2,1,0,0,0,0.0
2,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,73.6,0,0,0,0,0.4
3,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,73.6,0,0,0,0,0.0
4,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,73.4,1,0,0,0,-0.2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
577,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,75.6,1,0,1,1,0.2
578,0.0,1.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,75.6,1,0,1,1,0.0
579,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,75.0,0,0,1,1,-0.6
580,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,75.6,0,0,1,1,0.6


In [157]:
pd.concat([eat_concat.iloc[:,14:-6].drop(columns = "Etc. Comments"),eat_concat["Temp_diff"]],axis = 1)

SyntaxError: invalid syntax (<ipython-input-157-b5f6bd990b8f>, line 1)

In [156]:
eat_concat["Temp_diff"] = eat_concat["Temperature"].diff().fillna(0)

In [148]:
# import for Granger's Causality Test
from statsmodels.tsa.stattools import grangercausalitytests
granger_test = sm.tsa.stattools.grangercausalitytests(eat_concat.iloc[:,14:-5].drop(columns = "Etc. Comments"), maxlag=2, verbose=True)
granger_test


Granger Causality
number of lags (no zero) 1
ssr based F test:         F=2072.3632, p=0.0000  , df_denom=561, df_num=1
ssr based chi2 test:   chi2=2146.2443, p=0.0000  , df=1
likelihood ratio test: chi2=898.3981, p=0.0000  , df=1


ValueError: wrong shape for coefs

In [166]:
nobs = 20
df_train, df_test = pd.concat([eat_concat.iloc[:,14:-6].drop(columns = ["Etc. Comments","Temperature"]),eat_concat["Temp_diff"]],axis = 1)[0:-nobs], pd.concat([eat_concat.iloc[:,14:-6].drop(columns = ["Etc. Comments","Temperature"]),eat_concat["Temp_diff"]],axis = 1)[-nobs:]

In [ ]:
pd.set_option('display.max_columns', None)
eat_concat.iloc[:,14:-8].drop(columns = ["Etc. Comments","BT_SB_Location","GR_Location"])

In [163]:
# Augmented Dickey-Fuller Test (ADF Test)/unit root test
from statsmodels.tsa.stattools import adfuller
def adf_test(ts, signif=0.05):
    dftest = adfuller(ts, autolag='AIC')
    adf = pd.Series(dftest[0:4], index=['Test Statistic','p-value','# Lags','# Observations'])
    for key,value in dftest[4].items():
        adf['Critical Value (%s)'%key] = value
    print (adf)
    
    p = adf['p-value']
    if p <= signif:
        print(ts,f" Series is Stationary")
    else:
        print(ts,f" Series is Non-Stationary")
#apply adf test on the series
for i in df_train.columns:
    adf_test(df_train[i])

Test Statistic         -7.501681e+00
p-value                 4.237286e-11
# Lags                  7.000000e+00
# Observations          5.540000e+02
Critical Value (1%)    -3.442209e+00
Critical Value (5%)    -2.866771e+00
Critical Value (10%)   -2.569556e+00
dtype: float64
0      1.0
1      0.0
2      0.0
3      0.0
4      1.0
      ... 
557    1.0
558    0.0
559    0.0
560    0.0
561    1.0
Name: Saury, Length: 562, dtype: float64  Series is Stationary
Test Statistic           -4.085730
p-value                   0.001022
# Lags                    8.000000
# Observations          553.000000
Critical Value (1%)      -3.442230
Critical Value (5%)      -2.866781
Critical Value (10%)     -2.569561
dtype: float64
0      0.0
1      1.0
2      0.0
3      0.0
4      0.0
      ... 
557    0.0
558    1.0
559    0.0
560    0.0
561    0.0
Name: Blue_Runner, Length: 562, dtype: float64  Series is Stationary
Test Statistic           -4.899331
p-value                   0.000035
# Lags                

In [167]:
df_train.columns

Index(['Saury', 'Blue_Runner', 'Squid', 'Mackerel', 'Herring', 'Sardine',
       'Mazuri_Vitamins', 'Garlic', 'Salmon', 'Bonito', 'Bluefish', 'Mahi',
       'Goggle_Eye', 'Humbolt_Squid', 'GroupFeed', 'covid', 'light_training',
       'Varied_Target', 'Temp_diff'],
      dtype='object')

In [ ]:
import plotly.express as px
px.line(eat_concat, x = "Date", y = "fall")

In [168]:
model = VAR(df_train)
results = model.fit(maxlags=15, ic='aic')
results.summary()

  Summary of Regression Results   
Model:                         VAR
Method:                        OLS
Date:           Tue, 08, Dec, 2020
Time:                     09:30:25
--------------------------------------------------------------------
No. of Equations:         19.0000    BIC:                   -59.5809
Nobs:                     558.000    HQIC:                  -66.4909
Log likelihood:           6205.75    FPE:                1.64054e-31
AIC:                     -70.9188    Det(Omega_mle):     1.40714e-32
--------------------------------------------------------------------
Results for equation Saury
                        coefficient       std. error           t-stat            prob
-------------------------------------------------------------------------------------
const                      0.216917         0.115178            1.883           0.060
L1.Saury                   0.066312         0.062368            1.063           0.288
L1.Blue_Runner             0.063747     

In [173]:
model = VAR(df_train)
for i in [1,2,3,4,5,6,7,8,9]:
    result = model.fit(i)
    print('Lag Order =', i)
    print('AIC : ', result.aic)
    print('BIC : ', result.bic)
    print('FPE : ', result.fpe)
    print('HQIC: ', result.hqic, '\n')

Lag Order = 1
AIC :  -70.15352980711681
BIC :  -67.22074202797488
FPE :  3.411603618597557e-31
HQIC:  -69.00844126920896 

Lag Order = 2
AIC :  -70.63970376040255
BIC :  -64.91291598050374
FPE :  2.10586249985713e-31
HQIC:  -68.40353978825809 

Lag Order = 3
AIC :  -70.270892875203
BIC :  -61.742419316317644
FPE :  3.0755557655852224e-31
HQIC:  -66.94048031064932 

Lag Order = 4
AIC :  -70.91877990305
BIC :  -59.580899684476776
FPE :  1.6405429073985831e-31
HQIC:  -66.4909308050678 

Lag Order = 5
AIC :  -70.46852385820993
BIC :  -56.31348077449832
FPE :  2.658188910448702e-31
HQIC:  -64.94003540866836 

Lag Order = 6
AIC :  -69.86969379524696
BIC :  -52.88969609212192
FPE :  5.08092243258737e-31
HQIC:  -63.23734820387642 

Lag Order = 7
AIC :  -69.58436843287295
BIC :  -49.77158858134426
FPE :  7.24676371059659e-31
HQIC:  -61.84493284146838 

Lag Order = 8
AIC :  -69.73240890596682
BIC :  -47.07898337460517
FPE :  6.872030840447788e-31
HQIC:  -60.882635291813756 

Lag Order = 9
AIC : 

In [174]:
x = model.select_order(maxlags=12)
x.summary()

,AIC,BIC,FPE,HQIC
0,-56.18,-56.03,3.998e-25,-56.12
1,-69.95,-66.97*,4.190e-31,-68.78*
2,-70.38,-64.58,2.724e-31,-68.11
3,-70.02,-61.38,3.969e-31,-66.64
4,-70.68*,-59.22,2.076e-31*,-66.20
5,-70.26,-55.97,3.268e-31,-64.68
6,-69.69,-52.57,6.075e-31,-63.00
7,-69.43,-49.48,8.494e-31,-61.63
8,-69.61,-46.83,7.788e-31,-60.71
9,-69.27,-43.66,1.252e-30,-59.26


In [175]:
model_fitted = model.fit(4)
model_fitted.summary()

  Summary of Regression Results   
Model:                         VAR
Method:                        OLS
Date:           Tue, 08, Dec, 2020
Time:                     09:47:35
--------------------------------------------------------------------
No. of Equations:         19.0000    BIC:                   -59.5809
Nobs:                     558.000    HQIC:                  -66.4909
Log likelihood:           6205.75    FPE:                1.64054e-31
AIC:                     -70.9188    Det(Omega_mle):     1.40714e-32
--------------------------------------------------------------------
Results for equation Saury
                        coefficient       std. error           t-stat            prob
-------------------------------------------------------------------------------------
const                      0.216917         0.115178            1.883           0.060
L1.Saury                   0.066312         0.062368            1.063           0.288
L1.Blue_Runner             0.063747     

In [177]:
import random


In [182]:
random.randint(0,5, random_state = 42)

TypeError: randint() got an unexpected keyword argument 'random_state'

In [183]:
np.sum([1,5])

6

In [185]:
eat_concat["Total"].mean()

20.587628865979383

In [186]:
import statistics
statistics.variance(eat_concat["Total"])

92.7040119239846

In [29]:
import plotly.express as px
px.histogram(eat_concat, x = "Total")

In [188]:
px.histogram(eat_concat, x = "Total")

In [31]:
eat_concat["GroupFeed"]

0      1
1      1
2      0
3      0
4      1
      ..
577    1
578    1
579    0
580    0
581    1
Name: GroupFeed, Length: 582, dtype: int64